In [110]:
samples = [('D2FE28', 6),
           ('38006F45291200', 9),
           ('EE00D40C823060', 14),
           ('8A004A801A8002F478', 16),
           ('620080001611562C8802118E34', 12),
           ('C0015000016115A2E0802F182340', 23),
           ('A0016C880162017C3686B18A3D4780', 31), ]
my_in = '2056FA18025A00A4F52AB13FAB6CDA779E1B2012DB003301006A35C7D882200C43289F07A5A192D200C1BC011969BA4A485E63D8FE4CC80480C00D500010F8991E23A8803104A3C425967260020E551DC01D98B5FEF33D5C044C0928053296CDAFCB8D4BDAA611F256DE7B945220080244BE59EE7D0A5D0E6545C0268A7126564732552F003194400B10031C00C002819C00B50034400A70039C009401A114009201500C00B00100D00354300254008200609000D39BB5868C01E9A649C5D9C4A8CC6016CC9B4229F3399629A0C3005E797A5040C016A00DD40010B8E508615000213112294749B8D67EC45F63A980233D8BCF1DC44FAC017914993D42C9000282CB9D4A776233B4BF361F2F9F6659CE5764EB9A3E9007ED3B7B6896C0159F9D1EE76B3FFEF4B8FCF3B88019316E51DA181802B400A8CFCC127E60935D7B10078C01F8B50B20E1803D1FA21C6F300661AC678946008C918E002A72A0F27D82DB802B239A63BAEEA9C6395D98A001A9234EA620026D1AE5CA60A900A4B335A4F815C01A800021B1AE2E4441006A0A47686AE01449CB5534929FF567B9587C6A214C6212ACBF53F9A8E7D3CFF0B136FD061401091719BC5330E5474000D887B24162013CC7EDDCDD8E5E77E53AF128B1276D0F980292DA0CD004A7798EEEC672A7A6008C953F8BD7F781ED00395317AF0726E3402100625F3D9CB18B546E2FC9C65D1C20020E4C36460392F7683004A77DB3DB00527B5A85E06F253442014A00010A8F9106108002190B61E4750004262BC7587E801674EB0CCF1025716A054AD47080467A00B864AD2D4B193E92B4B52C64F27BFB05200C165A38DDF8D5A009C9C2463030802879EB55AB8010396069C413005FC01098EDD0A63B742852402B74DF7FDFE8368037700043E2FC2C8CA00087C518990C0C015C00542726C13936392A4633D8F1802532E5801E84FDF34FCA1487D367EF9A7E50A43E90'

hexa_bin = {
    '0': '0000',
    '1': '0001',
    '2': '0010',
    '3': '0011',
    '4': '0100',
    '5': '0101',
    '6': '0110',
    '7': '0111',
    '8': '1000',
    '9': '1001',
    'A': '1010',
    'B': '1011',
    'C': '1100',
    'D': '1101',
    'E': '1110',
    'F': '1111',
}


def get_input(s):
    return s.strip()


[get_input(s[0]) for s in samples]


['D2FE28',
 '38006F45291200',
 'EE00D40C823060',
 '8A004A801A8002F478',
 '620080001611562C8802118E34',
 'C0015000016115A2E0802F182340',
 'A0016C880162017C3686B18A3D4780']

In [111]:
def prob1(s):
    hexa = get_input(s)
    binary = get_binary(hexa)
    result = 0
    while True:
        if len(binary) > 7:
            binary, data = read_packet(binary)
            padding = len(binary) % 8
            binary = binary[padding:]
            result += data['version']
        else:
            break
    print(f'{hexa}: {result}')
    return result


def get_int(sub_bin):
    return int(sub_bin, 2)


def get_binary(hexa):
    result = ''
    for h in hexa:
        for i in hexa_bin[h]:
            result += i
    return result


def read_packet(binary):
    version = get_int(binary[:3])
    ptype = get_int(binary[3:6])
    if ptype == 4:
        new_binary, data = read_data(binary[6:])
    else:
        new_binary, data = read_operator(binary[6:])
        if 'version' in data:
            version += data['version']
    data['version'] = version
    data['type'] = ptype
    return new_binary, data


def read_data(binary):
    new_binary = binary
    result_data = {'content': []}
    while True:
        new_binary, data = read_data_chunk(new_binary)
        result_data['content'].append(data['data'])
        if data['finished']:
            break
    return new_binary, result_data


def read_data_chunk(new_binary):
    data = {
        'finished': new_binary[0] == '0',
        'data': new_binary[1:5]
    }
    # print(f'in read_chunk {new_binary} - {data}')
    return new_binary[5:], data


def read_operator(binary):
    if binary[0] == '0':
        new_binary, data = read_fixed_length(binary[1:])
    else:
        new_binary, data = read_num_packs(binary[1:])
    return new_binary, data


def read_fixed_length(binary):
    length = get_int(binary[:15])
    data = {'length': length, 'version': 0, 'content': []}
    new_binary = binary[length + 15:]
    our_binary = binary[15:length + 15]
    while our_binary:
        our_binary, data_pack = read_packet(our_binary)
        data['version'] += data_pack['version']
        data['content'].append(data_pack)
    return new_binary, data


def read_num_packs(binary):
    length = get_int(binary[:11])
    data = {
        'num_packages': length,
        'version': 0,
        'content': [],
    }
    new_binary = binary[11:]
    for i in range(length):
        new_binary, data_pack = read_packet(new_binary)
        data['version'] += data_pack['version']
        data['content'].append(data_pack)
    return new_binary, data


for sample in samples:
    test = prob1(sample[0])
    assert test == sample[1], f'Got {test}, expected {sample[1]}'
prob1(my_in)


D2FE28: 6
38006F45291200: 9
EE00D40C823060: 14
8A004A801A8002F478: 16
620080001611562C8802118E34: 12
C0015000016115A2E0802F182340: 23
A0016C880162017C3686B18A3D4780: 31
2056FA18025A00A4F52AB13FAB6CDA779E1B2012DB003301006A35C7D882200C43289F07A5A192D200C1BC011969BA4A485E63D8FE4CC80480C00D500010F8991E23A8803104A3C425967260020E551DC01D98B5FEF33D5C044C0928053296CDAFCB8D4BDAA611F256DE7B945220080244BE59EE7D0A5D0E6545C0268A7126564732552F003194400B10031C00C002819C00B50034400A70039C009401A114009201500C00B00100D00354300254008200609000D39BB5868C01E9A649C5D9C4A8CC6016CC9B4229F3399629A0C3005E797A5040C016A00DD40010B8E508615000213112294749B8D67EC45F63A980233D8BCF1DC44FAC017914993D42C9000282CB9D4A776233B4BF361F2F9F6659CE5764EB9A3E9007ED3B7B6896C0159F9D1EE76B3FFEF4B8FCF3B88019316E51DA181802B400A8CFCC127E60935D7B10078C01F8B50B20E1803D1FA21C6F300661AC678946008C918E002A72A0F27D82DB802B239A63BAEEA9C6395D98A001A9234EA620026D1AE5CA60A900A4B335A4F815C01A800021B1AE2E4441006A0A47686AE01449CB5534929FF567B9587C6A2

917

In [112]:
samples2 = {
    'D2FE28': 2021,
    'C200B40A82': 3,  # finds the sum of 1 and 2, resulting in the value 3.
    # finds the product of 6 and 9, resulting in the value 54.
    '04005AC33890': 54,
    # finds the minimum of 7, 8, and 9, resulting in the value 7.
    '880086C3E88112': 7,
    # finds the maximum of 7, 8, and 9, resulting in the value 9.
    'CE00C43D881120': 9,
    'D8005AC2A8F0': 1,  # produces 1, because 5 is less than 15.
    'F600BC2D8F': 0,  # produces 0, because 5 is not greater than 15.
    '9C005AC2F8F0': 0,  # produces 0, because 5 is not equal to 15.
    '9C0141080250320F1802104A08': 1,  # produces 1, because 1 + 3 = 2 * 2.
}


In [113]:
def prob2(s):
    hexa = get_input(s)
    binary = get_binary(hexa)
    parsed_data = []
    while True:
        if len(binary) > 7:
            binary, data = read_packet(binary)
            padding = len(binary) % 8
            binary = binary[padding:]
            parsed_data.append(data)
        else:
            break
    result = apply_instructions(parsed_data)[0]
    print(f'{hexa}: {result}')
    return result


def apply_instructions(instructions):
    result = []
    for instr in instructions:
        _t = instr['type']
        if _t == 4:
            data = ''
            for b in instr['content']:
                data += b
            result.append(int(data, 2))
            continue
        operands = apply_instructions(instr['content'])
        if _t == 0:
            result.append(sum(operands))
        elif _t == 1:
            result.append(product(operands))
        elif _t == 2:
            result.append(min(operands))
        elif _t == 3:
            result.append(max(operands))
        elif _t == 5:
            result.append(greater_than(operands))
        elif _t == 6:
            result.append(less_than(operands))
        elif _t == 7:
            result.append(equal_to(operands))
        else:
            assert False, instr
    return result


def product(operands):
    result = 1
    for op in operands:
        result *= op
    return result


def less_than(operands):
    return 1 if operands[0] < operands[1] else 0


def greater_than(operands):
    return 1 if operands[0] > operands[1] else 0


def equal_to(operands):
    return 1 if operands[0] == operands[1] else 0


for sample, expected in samples2.items():
    test = prob2(sample)
    assert test == expected, f'{sample}: Got {test}, expected {expected}'


D2FE28: 2021
C200B40A82: 3
04005AC33890: 54
880086C3E88112: 7
CE00C43D881120: 9
D8005AC2A8F0: 1
F600BC2D8F: 0
9C005AC2F8F0: 0
9C0141080250320F1802104A08: 1


In [114]:
prob2(my_in)


2056FA18025A00A4F52AB13FAB6CDA779E1B2012DB003301006A35C7D882200C43289F07A5A192D200C1BC011969BA4A485E63D8FE4CC80480C00D500010F8991E23A8803104A3C425967260020E551DC01D98B5FEF33D5C044C0928053296CDAFCB8D4BDAA611F256DE7B945220080244BE59EE7D0A5D0E6545C0268A7126564732552F003194400B10031C00C002819C00B50034400A70039C009401A114009201500C00B00100D00354300254008200609000D39BB5868C01E9A649C5D9C4A8CC6016CC9B4229F3399629A0C3005E797A5040C016A00DD40010B8E508615000213112294749B8D67EC45F63A980233D8BCF1DC44FAC017914993D42C9000282CB9D4A776233B4BF361F2F9F6659CE5764EB9A3E9007ED3B7B6896C0159F9D1EE76B3FFEF4B8FCF3B88019316E51DA181802B400A8CFCC127E60935D7B10078C01F8B50B20E1803D1FA21C6F300661AC678946008C918E002A72A0F27D82DB802B239A63BAEEA9C6395D98A001A9234EA620026D1AE5CA60A900A4B335A4F815C01A800021B1AE2E4441006A0A47686AE01449CB5534929FF567B9587C6A214C6212ACBF53F9A8E7D3CFF0B136FD061401091719BC5330E5474000D887B24162013CC7EDDCDD8E5E77E53AF128B1276D0F980292DA0CD004A7798EEEC672A7A6008C953F8BD7F781ED00395317AF0726E34021

2536453523344

In [115]:
for s in samples:
    prob2(s[0]) 

D2FE28: 2021
38006F45291200: 1
EE00D40C823060: 3
8A004A801A8002F478: 15
620080001611562C8802118E34: 46
C0015000016115A2E0802F182340: 46
A0016C880162017C3686B18A3D4780: 54
